# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [6]:
city_weather_data = pd.read_csv("Data_CSV/city_weather_data.csv", encoding="ISO-8859-1")
city_weather_data.head()


,Unnamed: 0,City,Max Temperature,Humidity,Cloudiness,Wind Speed,Latitude,Longitude
0,0,Vaini,78.80,94,75,14.97,-21.2000,-175.2000
1,1,Khatanga,-29.72,79,0,10.65,71.9667,102.5000
2,2,Talcahuano,62.60,82,75,6.91,-36.7167,-73.1167
3,3,Tasiilaq,26.60,46,0,18.41,65.6145,-37.6368
4,4,SÄmbhar,53.60,62,0,3.20,26.9167,75.2000


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [7]:
locations = city_weather_data[['Latitude', 'Longitude']]
weight = city_weather_data['Humidity']

In [9]:
layout = {'width': '900px',
    'height': '700px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'}

figure = gmaps.figure(layout = layout)

figure = gmaps.figure(center=(40.0, -5.0), zoom_level=2) 
max_intensity = np.max(weight)

heat_layer = gmaps.heatmap_layer(locations, weights = weight, dissipating=False, max_intensity=100, point_radius=3)

figure.add_layer(heat_layer)

figure


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
